# Getting Weather Data using a web API
## [Weather API](https://www.weatherapi.com)
---

In [1]:
import requests
from datetime import datetime, timedelta
import json
import csv

# date time -> to find the current date
# time delta -> to find the time difference

In [2]:
def weather(api_key, city, start_date, end_date):
    base_url = f"http://api.weatherapi.com/v1/history.json?key={api_key}&q={city}&dt={start_date}&end_dt={end_date}"
    print(base_url)

    response = requests.get(base_url)
    print("Status Code =", response.status_code)
    if response.status_code == 200:
        data = response.json()

        # Storing the response in a JSON file
        with open("response.json", "w") as file:
            json.dump(data, file)

        forecast_days = data["forecast"]["forecastday"]

        # Writing the column_names in the csv file
        with open("weather_data.csv", mode="w") as file:
            writer = csv.writer(file)
            writer.writerow(["date", "temperature", "humidity", "weather_description"])

        for forecast in forecast_days:
            date = forecast["date"]
            temperature = forecast["day"]["avgtemp_c"]
            humidity = forecast["day"]["avghumidity"]
            weather_description = forecast["day"]["condition"]["text"]

            print("-----------------===-----------------")
            print(f"Weather in {city} on {date}:")
            print(f"Temperature: {temperature}°C")
            print(f"Humidity: {humidity}%")
            print(f"Weather Description: {weather_description}")
            print("-----------------===-----------------")

            with open("weather_data.csv", mode="a") as file:
                writer = csv.writer(file)
                writer.writerow([date, temperature, humidity, weather_description])

    else:
        print("Error in fetching data :", response.content)

In [3]:
key = "bd78dba193bc42238cb94729232106"
city = "Udupi"

# format for date is "YYYY-MM-DD"
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")

weather(key, city, start_date, end_date)

http://api.weatherapi.com/v1/history.json?key=bd78dba193bc42238cb94729232106&q=Udupi&dt=2023-05-28&end_dt=2023-06-27
Status Code = 200
-----------------===-----------------
Weather in Udupi on 2023-05-28:
Temprature: 30.7°C
Humidity: 69.0%
Weather Description: Thundery outbreaks possible
-----------------===-----------------
-----------------===-----------------
Weather in Udupi on 2023-05-29:
Temprature: 29.8°C
Humidity: 70.0%
Weather Description: Patchy light rain with thunder
-----------------===-----------------
-----------------===-----------------
Weather in Udupi on 2023-05-30:
Temprature: 30.1°C
Humidity: 73.0%
Weather Description: Thundery outbreaks possible
-----------------===-----------------
-----------------===-----------------
Weather in Udupi on 2023-05-31:
Temprature: 28.6°C
Humidity: 70.0%
Weather Description: Patchy rain possible
-----------------===-----------------
-----------------===-----------------
Weather in Udupi on 2023-06-01:
Temprature: 28.7°C
Humidity: 77

---
        ## Trying to predict the weather data by applying a ML model
---

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("weather_data.csv")
df.isnull().sum()

date                   0
temperature            0
humidity               0
weather_description    0
dtype: int64

In [3]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df["weather_description"] = le.fit_transform(df["weather_description"])
df

,date,temperature,humidity,weather_description
0,2023-05-28,30.7,69.0,5
1,2023-05-29,29.8,70.0,3
2,2023-05-30,30.1,73.0,5
3,2023-05-31,28.6,70.0,4
4,2023-06-01,28.7,77.0,4
5,2023-06-02,29.1,75.0,5
6,2023-06-03,30.4,71.0,2
7,2023-06-04,29.9,70.0,0
8,2023-06-05,29.3,73.0,4
9,2023-06-06,28.6,75.0,1


In [4]:
X = df.drop(["date", "weather_description"], axis=1)
y = df["weather_description"]

In [5]:
from sklearn.model_selection import train_test_split as tts

X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

In [6]:
from sklearn.tree import DecisionTreeClassifier

dfc = DecisionTreeClassifier()
dfc.fit(X_train, y_train)

DecisionTreeClassifier()

In [7]:
dfc.score(X_test, y_test)

0.42857142857142855